In [1]:
# reload imports.
%load_ext autoreload
%autoreload 2

### Get a sample size

We can't test on all of them. Let's find an acceptable sample size and isolate some
We'll pull from geometry - their solution had the most trouble w/ geometry, and lots of variance on it.

In [2]:
import os
import json

# Gemotry - t
folder_path = "./MATH/test/geometry/"
json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]

# Store each in list
json_objects = []
for file in json_files:
    file_path = os.path.join(folder_path, file)
    with open(file_path, 'r') as f:
        json_data = json.load(f)
        json_data["file_path"]=file_path # Add file path so we can keep track of them easily
        json_objects.append(json_data)

len(json_objects)

479

Let's only do the hard ones - these are the ones they test in the paper

In [3]:
filtered_json_objects = [obj for obj in json_objects if obj.get('level') == 'Level 5']
print(filtered_json_objects[:1])
print(len(filtered_json_objects))


[{'problem': "A solid $5\\times 5\\times 5$ cube is composed of unit cubes. Each face of the large, solid cube is partially painted with gray paint, as shown. [asy]\n\nfill((0,0)--(0,1)--(1,1)--(1,0)--cycle,gray);\n\nfill((0,4)--(0,5)--(1,5)--(1,4)--cycle,gray);\n\nfill((4,1)--(5,1)--(5,0)--(4,0)--cycle,gray);\n\nfill((1,2)--(2,2)--(2,1)--(1,1)--cycle,gray);\n\nfill((2,2)--(3,2)--(3,1)--(2,1)--cycle,gray);\n\nfill((3,2)--(4,2)--(4,1)--(3,1)--cycle,gray);\n\nfill((1,3)--(2,3)--(2,2)--(1,2)--cycle,gray);\n\nfill((3,3)--(4,3)--(4,2)--(3,2)--cycle,gray);\n\nfill((1,4)--(2,4)--(2,3)--(1,3)--cycle,gray);\n\nfill((2,4)--(3,4)--(3,3)--(2,3)--cycle,gray);\n\nfill((3,4)--(4,4)--(4,3)--(3,3)--cycle,gray);\n\nfill((4,5)--(5,5)--(5,4)--(4,4)--cycle,gray);\n\ndraw((0,0)--(0,1)--(1,1)--(1,0)--(0,0),rgb(0,0,0));\n\ndraw((0,1)--(0,2)--(1,2)--(1,1),rgb(0,0,0));\n\ndraw((0,2)--(0,3)--(1,3)--(1,2),rgb(0,0,0));\n\ndraw((0,3)--(0,4)--(1,4)--(1,3),rgb(0,0,0));\n\ndraw((0,4)--(0,5)--(1,5)--(1,4),rgb(0,0,0));\

There is 132. HOw many of these at random do we need to pick to get a reasonable estimate?

We can use the finite sample size forumla



In [4]:
import math


# Copied from equation here: https://online.stat.psu.edu/stat415/lesson/6/6.3
def calculate_sample_size(N, p_hat, Z, E):
    m = (Z**2 * p_hat * (1 - p_hat)) / (E**2)
    n = m / (1 + ((m - 1) / N))

    return math.ceil(n)

N = len(filtered_json_objects) # 132 for geometry
p_hat = 0.5      # Estimated proportion - unknown so use .50
Z = 1.96         # Z-score for 95% confidence level
E = 0.10         # Desired margin of error 

# Calculate the required sample size
required_sample_size = calculate_sample_size(N, p_hat, Z, E)
print(f"Required sample size: {required_sample_size}")


Required sample size: 56


Even at a 10% margin of error we would still need to run our test 56 times.. that'll cost:

In [5]:
# 20-35 cents per run
print(f"${required_sample_size * ((.20+.35)/2):.2f}")

$15.40


Such is the price we pay for progress. 

Let's isolate 56 random from the filtered_json_objects, and save this

In [6]:
import random

random.seed(7) # lucky number 7

length = len(filtered_json_objects)

sampled = random.sample(filtered_json_objects, required_sample_size)

print(len(sampled))
print(sampled[0]["file_path"])

56
./MATH/test/geometry/393.json


4 objects at random

### Experiment 1

Let's take 5 for testing. And do a run of 3. This way we can compare the variances. 
We're going to log them and then verify them manually

ok.. what do I want to do now?? Let's see.. I want to make a space to save the logs of each run, and then run on each one of these problems. The name of the log should include the file path. 

I'll have to change my logging code to log to somewhere specific. hmm...

Then I want to try running it on how it is now.
Then I want to run it w/ only using the code assistant for the executor. Maybe this will help reduce costs and not be that much more :)
I can also try running w/ gpt 4o-mini on some of the simple tasks. e.g. summarizing. 

Then I can see which ones it's failing on. And see how they're both doing. And then we can go problem by problem and see what tweaks can fix things


In [7]:
import random

random.seed(10)
samples = random.sample(filtered_json_objects, 5)
samples[0].keys()

dict_keys(['problem', 'level', 'type', 'solution', 'file_path'])

#### Main 1

In [8]:
%%script echo skipping
# Unfortunately this clears the output, but used to avoid running this (40 minute) process each time


from utils.custom_logger import CustomLogger
from utils.create_assistant import create_agents_and_thread
from chains.main1.main import main

# Use the same agents and threads to (ideally) limit code sessions
coding_assistant, coding_thread = create_agents_and_thread()

# Do 3 runs
for i in range(3):
    # Do 5 problems
    for j, problem in enumerate(samples):
        CustomLogger.update_path(f"run-{i}/problem-{j}")
        CustomLogger.default_log("Problem File Path", problem["file_path"])

        max_times_mining_new = 1  # The upper limit of the mining times
        question = problem["problem"]
        main(question, max_times_mining_new, coding_assistant, coding_thread)


skipping


Observations:

New ideas:
- Seems like we fail on the steps being wrong - try checking these & improving the prompt to not calculate yet
1. Add a verify_steps function, and improve the steps prompt

#### Main 2

In [9]:
%%script echo skipping
import re
from utils.custom_logger import CustomLogger
from utils.create_assistant import create_agents_and_thread
from chains.main2.main import main # THIS IS THE ONLY DIFFERENCE


# Use the same agents and threads to (ideally) limit code sessions
coding_assistant, coding_thread = create_agents_and_thread()

# Make nested arr
verify_arr = [f"Problem {i} -> " for i,_ in enumerate(samples)]
# 3 runs of 5 problems
for i in range(3):
    for j, problem in enumerate(samples):
        CustomLogger.update_path(f"run-{i}/problem-{j}")
        CustomLogger.default_log("Problem File Path", problem["file_path"])

        max_times_mining_new = 1  # The upper limit of the mining times
        question = problem["problem"]
        our_answer = main(question, max_times_mining_new, coding_assistant, coding_thread)


        # TODO: fix the capture group - currently it says everything is incorrect
        # Validate
    
        # This long regex *should* recursively balance the ending parentheses
        actual_answer = re.search(r'\\boxed{((?:[^{}]+|{(?:[^{}]+|{[^{}]*})*})*)}', problem["solution"]).group(1)
        if actual_answer:
            CustomLogger.default_log("Actual Answer", actual_answer)
            if True:
                CustomLogger.default_log("Correct")
                verify_arr[j]+= "correct "
        
        else:
            CustomLogger.default_log("Incorrect")
            verify_arr[j]+= "incorrect "
            

# Log verify arr -> validation
CustomLogger.update_path("validation")
CustomLogger.default_log("Results", *verify_arr)


skipping


Observations:

- The re-assessment of steps we added is good. But for instance in run 0 problem 3, the reassessment fails twice in a row so we just kep going. We need to explain why it fails and then add that context to the step creator.
1. To do this we can ask it to explain why its failing - let's do a schema w/ true false and reason. Since schema doesn't generate intermediate, lets do two calls, one normal and one schema. 

- Also in problem 3, I think we end up with conditions that I think were contradictory. To improve this we can do two things. 
1. Generate less conditions each time, and then increase the amount of mining loops we do
2. After finishing all the mining, ask if any conditions are contradictory. If they are, return those indices



Changes for main 3-
1. added reasoning for failed steps, to be passed to step creator in the form of optional_comment
2. Updated verify steps to use schema & tweaked prompt
3. Limit conditions to generate each time both by asking GPT and through slicing

4. Experimented w/ a double check of conditions, but got into a bit of a snarl. If two conditions are contradictory, how do we know which is true? We have to re-wind a lot, and it gets confusing. This might not work without a drastic overhaul of the system.



#### Main 3

In [10]:
%%script echo skipping
import re
from utils.custom_logger import CustomLogger
from utils.create_assistant import create_agents_and_thread
from chains.main3.main import main # THIS IS THE ONLY DIFFERENCE


# Use the same agents and threads to (ideally) limit code sessions
coding_assistant, coding_thread = create_agents_and_thread()

# This long regex *should* recursively balance the ending parentheses
def get_boxed(problem):
    return re.search(r'\\boxed{((?:[^{}]+|{(?:[^{}]+|{[^{}]*})*})*)}', problem["solution"]).group(1)

MAX_MINING = 3 # Bumped this up!


generated_answers = [f"Problem {i} -> " for i,_ in enumerate(samples)]
actual_answers= [f"Problem {i} -> {get_boxed(problem)}" for i, problem in enumerate(samples)]
# 3 runs of 5 problems
for i in range(3):
    for j, problem in enumerate(samples):
        CustomLogger.update_path(f"run-{i}/problem-{j}")
        CustomLogger.default_log("Problem File Path", problem["file_path"])

        question = problem["problem"]
        our_answer = main(question, MAX_MINING, coding_assistant, coding_thread)
        generated_answers[j]+= f"{our_answer}, "

        CustomLogger.default_log("Correct Answer:", get_boxed(problem))
            
# TODO: It would be cool to auto compare them - however the actual answers are given in formula form, e.g. 3/5. 
CustomLogger.update_path("validation")
CustomLogger.default_log("Generated", *generated_answers)
CustomLogger.default_log("Actual", *actual_answers)


skipping


**Observations from main 3**

- There's at least one instance of the executor failing to execute correctly. Lets add in a quick sanity check after the executor finishes. This sanity check, if it fails, triggers a re-run of the execution plus a reason (DO and DO NOT) for why it failed

- The logs are getting way too long. I need to trim them down. I'll probably switch to logging in main. I want to keep the full logs somewhere though, I just also want to have non-full logs.

- I also want to switch to async since right now it takes forever..


#### Main 4

In [11]:
%%script echo skipping
import anyio
import re
from chains.main4.main import main4
from utils.create_assistant import create_agents_and_thread, WorkSpace
from utils.custom_logger import CustomLogger
from utils.async_logger import AsyncLogger
from contextlib import asynccontextmanager
from collections import deque


class WorkSpacePool():
    def __init__(self, workspaces: list[WorkSpace]):
        self.workspaces = deque(workspaces)
        self.semaphore = anyio.Semaphore(len(workspaces))

    @asynccontextmanager
    async def get_space(self):
        async with self.semaphore:
            workspace = self.workspaces.popleft()
            try:
                yield workspace
            finally:
                self.workspaces.append(workspace)

def get_boxed(problem):
    return re.search(r'\\boxed{((?:[^{}]+|{(?:[^{}]+|{[^{}]*})*})*)}', problem["solution"]).group(1)

MAX_MINING = 3 
generated_answers = [f"Problem {i} -> " for i, _ in enumerate(samples)]
actual_answers= [f"Problem {i} -> {get_boxed(problem)}" for i, problem in enumerate(samples)]
failed_samples = []
results = []  

# Async wrapper - modifies global results to get the results out of the async
async def wrapper(pool: WorkSpacePool, question, log_path, j):
    async with pool.get_space() as workspace:
        try:
            # Attempt to get result from main4
            result = await main4(question, MAX_MINING, workspace.assistant, workspace.thread, log_path)
            results.append(result)
            generated_answers[j] += f"{result}, " #TODO: the ordering in the generated answers may be incorrect b.c. async 
        except Exception as e:
            # Handle main4 failure, log it, and store the problem for later re-run
            CustomLogger.print(f"Error on problem {j} in run {log_path}: {e}")
            failed_samples.append((log_path, question, j))  #TODO: do something w/ the failed samples
            AsyncLogger.add_message(log_path, "Error", str(e))
            AsyncLogger.flush_one(log_path)

async def main():
    #TODO: switch to individual timedelta for each run...
    # Record start time so we can get timedelta for each run.
    CustomLogger.start_watch() 

    # Limit the number of concurrent tasks to avoid rate limits & code session charges :)
    workspaces = [create_agents_and_thread() for _ in range(5)]
    pool = WorkSpacePool(workspaces)

    async with anyio.create_task_group() as tg:
       for i in range(3):
            for j, problem in enumerate(samples):
                log_path = f"run-{i}/problem-{j}"
                AsyncLogger.add_message(log_path, "Problem File Path", problem["file_path"])
                question = problem["problem"]
                tg.start_soon(wrapper, pool, question, log_path, j) 
    
    CustomLogger.print("All answers collected.")

    CustomLogger.update_path("validation")
    CustomLogger.default_log("Generated", *generated_answers)
    CustomLogger.default_log("Actual", *actual_answers)


try:
    await main()
except Exception as main_exception:
    CustomLogger.print(f"Exception in main: {main_exception}")


skipping


Awesome! I was able to make it asynchronous. I have a couple ideas of what to do next.

1. My first is that I think the way I've been validating is less than ideal. Instead of running on 15 I can instead run the same problem ~5 or so times. And I can aim for getting the same problem correct every time. If I can improve its percentage of being correct, I can track progress better than 5x3. I can even run the same problem 10 times for the same cost as before. 

2. Next - I started thinking about the idea behind this process in the first place. The idea is to break it into smaller logical chunks and verify those. However, we get incorrect sections of logic. Sometimes we have an incorrect formula, or its applied incorrectly, or sometimes conditions contradict each other.

It would be good to be able to trace back our steps in logic - see what conditions each other condition is built off of.
- We could try generating one condition at a time, and see what that looks like. And then build a history. This history is important so we can roll back and check. And periodically we can walk through and check our work. 

3. When we write the steps for the end, it would be good to standardize the language. More than that, it would be good to standardize what each step uses. Something like Use [technique or condition] along with [conditions] to calculate [step answer] which is needed because [reason]

4. It would be nice to have mini objectives. I'm looking at the solution for one of the problems written out. It's something like:
    - Draw diagram. Triangle is equilateral. Because of that, it's area is <area formula>
    - Draw on the diagram to show this given representation. *notice* that this makes 3 new triangles. We can sum their areas to find the total area. This gives us an equation.
    
    Each step has an objective, or a couple. It's setting up the problem for us. The condition gathering tries to mimic this process - each condition is supposed to be basically a mini step. However, we generate multiple at a time, and we don't differentiate which ones came first. So we treat them all with the same level. We need a chronological order. This way we can see what leads to other conditions.

5. Are the original extracted conditions always good? Can we get some way of getting pretty much always the same extracted conditions from the problem? If we run it e.g. 5 times and pull them together can we achieve this? We can also consider going back and getting more conditions from the problem - sometimes we will hit steps that will cause us to need to know more problem info.
    - Hmm.. so its impossible to know ahead of time exactly which conditions will be needed. We can pull some, but I think as we go through the problem we will want to revise the problem statement and get new conditions. We should do this in general I think - revising early parts to see what else we can get from it. Again, we need the history. I think to have effective history we need a specific structure - like reason, calculation, assumption.
    

Summary of these thoughts and implementation:
- run on 1 problem 10-15 times.
- chronological history w/ standardized language and 'steps'
- Can the standardized language/steps include some sort of mini objective?
- As the problem develops and our history/steps gets longer, we may realize that we need to go back and check a condition that didn't seem important before - e.g. if a triangle is equilateral. How can we handle this? more importantly, *where* should be this be stored? The 'chronological' history will be used to create final steps, so it should be placed in the history somewhere where it doesn't affect things. This requires us to re-check unfortunately..

### New Tactic!
Based on the above ideas..

Let's walk through making a new process in a notebook format. The functions used will be stored in new_stuff to avoid too much clutter



In [12]:
# Starting with question 0 since it's not too hard. We normally get it right most times.
# Let's see if we can get it right every time, & use it to refine our approach


question = samples[0]['problem']
print(question)


Point $P$ is inside equilateral triangle $ABC$ such that the altitudes from $P$ to $\overline{AB}$, $\overline{BC}$, and $\overline{CA}$ have lengths 5, 6, and 7 respectively.  What is the area of triangle $ABC$?


Working Process:

Stored:
- Given conditions *directly from problem* - can be updated
- Proven mathematical relationships - written as formulas
- Problem relationships - written without math, just words
- Intermediate math relationships - numerical relationships between parts of the problem

1. Get given conditions from problem

2. Identify a directly related mathematical relationship from some number of conditions. Something like, because equilaterial triangle, we can use area formula a=s^2...
    1. check this and make sure its a real condition (for now just ask gpt. Later we may want to test w/ code)
    2. If it's not, add to list of false conditions? *TODO: think about how to do this without polluting*

4. Identify a problem solving *method* that could lead to some further knowledge about the problem. This could be, e.g. we can make three smaller triangles, and we know their areas can be calculated, so this wil give us a formula
    1. check and make sure it makes sense to try - will lead to more givens, and has a good reason aligned w/ the objectives list.
    2. If not, keep trying. Somehow retain history to avoid duplicating and getting stuck.

5. Once we have our method, we need to check the assumptions in it. 
    1. We need to verify the important mathematical relationships (adding them to our relationship list) - e.g. the area of a triangle equals 1/2 * bh.
    2. We need to verify the specific problem relationships - e.g. the area of all the smaller triangles will sum to the larger one BECAUSE they are contained inside it.

6. Finally, we implement our method, and see what new things we get from it. It should give use intermediate math relationships, basically just formulas with numbers plugged in. This will involve setting up formulas, then using code to calculate out equations

7. Now we add the new things to our lists. The method gets added to our history? Hmm.. not sure what shoulod be stored in chronological history and what should be stored in just lists


*for now we are going to move what used to be #3 to the end*

3. Did this create a sub objective? If so, add it to objectives list. e.g. find side length FOR REASON can use formula to find goal.

In [13]:
# Sample logging code:
# from utils.helpers import conditions_objectives_to_string
# c_str, o_str = conditions_objectives_to_string(conditions, objectives)
# AsyncLogger.add_message(
#     log_path, "---\nHistory Snapshot:", "Got givens (conditions, objective)", c_str, o_str, "\n"
# )

# AsyncLogger.flush()

In [14]:
# 1. Conditions from question
from new_stuff.all import givens_from_question

log_path = 'problem0'
storage= {}

conditions, objectives = await givens_from_question(question)
print(conditions)
print(objectives)

storage["conditions"] = conditions
storage["objectives"] = objectives

Future exception was never retrieved
future: <Future finished exception=BrokenPipeError(32, 'Broken pipe')>
Traceback (most recent call last):
  File "/Users/dmccanns/miniforge3/envs/macm/lib/python3.10/asyncio/unix_events.py", line 676, in write
    n = os.write(self._fileno, data)
BrokenPipeError: [Errno 32] Broken pipe


['Point P is inside equilateral triangle ABC.', 'The altitude from point P to side AB has a length of 5.', 'The altitude from point P to side BC has a length of 6.', 'The altitude from point P to side CA has a length of 7.', 'Triangle ABC is equilateral, implying all its internal angles are 60 degrees.', 'The three sides of triangle ABC are of equal length.', 'The location of point P is such that it forms perpendiculars to all three sides AB, BC, and CA.', 'Altitude from P mean the three lines from P to AB, BC, and CA intersect perpendicularly at those lines.']
Determine the area of equilateral triangle ABC.


In [15]:
#2. Math relationships + verification
from new_stuff.all import relationships_from_existing, verify_relationship

relationships = await relationships_from_existing(conditions, objectives)
print(relationships)

new_relationships = []
# Check that it exists - for now just ask gpt. If this is failing a lot then we can use coder
for r in relationships:
    ## TODO: fix relationships.value[2]. Move this back into the relationships_from_existing function. 
    verification = await verify_relationship(r, conditions, objectives)
    print(verification)

    if verification.accept:
        new_relationships.append(r)

storage["relationships"] = new_relationships

[MathRelationship(conditions=[1, 2, 3, 4], relationship='Area = (1/2) [d1 * PA + d2 * PB + d3 * PC] = (1/2)[a*5 + a*6 + a*7] = 9*a', reason='This approach involves calculating the area of ΔABC using its subareas formed by perpendiculars from P to its sides, characterized by a ratio formula concerning altitudes.'), MathRelationship(conditions=[5, 6], relationship='Area = (sqrt(3)/4)*a^2', reason='Since ΔABC is an equilateral triangle, its area directly utilizes this formula, depending only on the length of its side a.')]
relevant=False proven=False accept=False
relevant=True proven=True accept=True


In [16]:
# 3. Sub-condition check (backwards problem solving) ??

In [19]:
#4. Find a method to get more information
from new_stuff.all import find_method

method = await find_method(question, storage["relationships"], storage["conditions"], storage["objectives"])
print(method)



method="### Methodology: Using Viviani's Theorem in Equilateral Triangle \n\n**Reasoning:**\nViviani's Theorem states that in an equilateral triangle, the sum of the perpendicular distances from a point inside the triangle to the three sides is equal to the altitude of the triangle. This theorem allows us to compute the side of the equilateral triangle and further infer its area using the known formula for the area of an equilateral triangle.\n\n**Steps to Approach:**\n1. **Sum of Altitudes from Point P:** \n    - Sum the given perpendiculars from point \\(P\\) to the sides: \n      \\(h_{PA} + h_{PB} + h_{PC} = 5 + 6 + 7 = 18\\).\n\n2. **Relate to the Altitude of Triangle ABC:** \n    - According to Viviani’s Theorem, this sum (18) is equal to the altitude \\(H\\) of the equilateral triangle\\(ABC\\). Therefore, \\(H = 18\\).\n\n3. **Use the Formula for Altitude in an Equilateral Triangle:** \n   - The altitude \\(H\\) of an equilateral triangle with side \\(a\\) is given by the formu

Hmmm... I'm not finished with this method but I think I'm under-utilizing the knowledge of the LLM. I also think I'm going about this process backwards. Instead I could ask it what the first step? is and kind of iterate recursively breaking it down smaller and smaller. 

What are the problems here?
    - I'm afraid it will try and do the whole thing in one shot. And we know this doesnt work
    - But if I ask it to just do one step its harder

I want to somehow like ask it for its step, and then recursively break this down and check each step. This way I can use it's problem solving ability.


Maybe I should try asking it to solve to the end to start, and have it return a list of steps. Then I can go back and see how many are correct?
We can limit this by only having it create max 5 steps.

We have it find the first step that is incorrect. the first mistake. Then we go from the beginning to there, and each step we check:
1. Should we break this further into steps? is it detailed?
    1. If so, repeat this process and recurse
2. If not, then we analyze this step, and extract the following things
    - Proven mathematical relationships - written as formulas
    - Problem relationships - written without math, just words. Some sort of logical condition in the project
    - Intermediate math relationships - numerical relationships between parts of the problem. Only valid in this context. Uses a math formula and some specific knowledge. Break up into a proven math rel and a problem rel

3. We need to verify the above. 
    - Proven math relations can be verified by asking
    - Problem relationships verified by asking as well
    - Intermediate math relationships should have any math relations verified, then any problem relationships verified, then finally they should be executed and verified with code

4. Now the step is good to go. We can re-write it to show the verified above things. We can also store the proven relationships and problem relationships seperately. 

5. Now if at any point in the larger step we had a problem, we are going to have to recalculate the rest of the steps. If it's the same we can continue through the steps, but if its different it may change the next ones.
    - I think this is fine. We need some way of error correcting, and propogating that correction

### Tactic 2:
1. Ask what it would like to do - what is the first step, given what we have.
2. Then analyze the step it returns - what is the rationale behind it? What outside theorems does it use? What new assumptions does it make?
3. Check each assumption individually, and iterate until we get a good first step.
4. Then do this step, update the knowns, and then repeat the process until we get the objective